<a href="https://colab.research.google.com/github/wbyche/awesome-python/blob/master/CLIP_based_NSFW_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install clip-anytorch autokeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 9.7 MB/s 
     |████████████████████████████████| 162 kB 35.1 MB/s 
     |████████████████████████████████| 53 kB 1.0 MB/s 
     |████████████████████████████████| 135 kB 46.8 MB/s 
     |████████████████████████████████| 1.6 MB 31.9 MB/s 


In [ ]:
import os
import numpy as np


def load_safety_model(clip_model="ViT-L/14"):
    """load the safety model"""
    import autokeras as ak  # pylint: disable=import-outside-toplevel
    from tensorflow.keras.models import load_model  # pylint: disable=import-outside-toplevel

    cache_folder = "./NSFW-cache"

    if clip_model == "ViT-L/14":
        model_dir = cache_folder + "/clip_autokeras_binary_nsfw"
        dim = 768
    else:
        raise ValueError("Unknown clip model")
    if not os.path.exists(model_dir):
        os.makedirs(cache_folder, exist_ok=True)

        from urllib.request import urlretrieve  # pylint: disable=import-outside-toplevel

        path_to_zip_file = cache_folder + "/clip_autokeras_binary_nsfw.zip"
        if clip_model == "ViT-L/14":
            url_model = "https://raw.githubusercontent.com/LAION-AI/CLIP-based-NSFW-Detector/main/clip_autokeras_binary_nsfw.zip"
        elif clip_model == "ViT-B/32":
            url_model = (
                "https://raw.githubusercontent.com/LAION-AI/CLIP-based-NSFW-Detector/main/clip_autokeras_nsfw_b32.zip"
            )
        else:
            raise ValueError("Unknown model {}".format(clip_model))  # pylint: disable=consider-using-f-string
        urlretrieve(url_model, path_to_zip_file)
        import zipfile  # pylint: disable=import-outside-toplevel

        with zipfile.ZipFile(path_to_zip_file, "r") as zip_ref:
            zip_ref.extractall(cache_folder)

    loaded_model = load_model(model_dir, custom_objects=ak.CUSTOM_OBJECTS)

    return loaded_model

safety_model = load_safety_model()



test_embeddings = np.random.rand(10**2, 768).astype("float32")

nsfw_values = safety_model.predict(test_embeddings, batch_size=test_embeddings.shape[0])




print(nsfw_values)

1/1 [==============================] - 3s 3s/step
[[1.40391360e-03]
 [4.65177266e-07]
 [6.22546773e-13]
 [2.45058000e-05]
 [1.18567839e-22]
 [5.01522344e-08]
 [9.99048293e-01]
 [4.53848435e-21]
 [2.07128937e-17]
 [1.06230339e-16]
 [1.25147226e-09]
 [1.74731277e-22]
 [2.92592200e-20]
 [2.34098543e-20]
 [2.75839247e-12]
 [3.07419807e-13]
 [3.29380383e-23]
 [3.27616643e-19]
 [1.98920329e-12]
 [1.11933776e-04]
 [5.24739806e-18]
 [9.88017987e-08]
 [2.22035823e-03]
 [1.03854238e-24]
 [9.99634922e-01]
 [2.08311424e-11]
 [1.42098804e-08]
 [3.00928369e-18]
 [3.49855609e-14]
 [5.35455153e-17]
 [1.12931227e-11]
 [6.32023676e-13]
 [1.72033950e-07]
 [8.84789294e-23]
 [9.89841580e-01]
 [1.70829226e-18]
 [2.56863758e-13]
 [6.00705818e-10]
 [5.26834876e-14]
 [4.75363120e-33]
 [7.99154700e-30]
 [8.44431122e-12]
 [2.70753138e-21]
 [7.57822692e-02]
 [5.20148161e-14]
 [2.91664992e-03]
 [4.89236436e-06]
 [1.00040234e-08]
 [3.10989208e-08]
 [3.09039366e-25]
 [7.55148168e-15]
 [2.62209202e-18]
 [9.84648228e-

In [ ]:
import torch
import clip
from PIL import Image
!wget https://www.damenmode-quelle.de/fotky421/fotos/_vyr_6097Sexy-Push-Up-Bikini-Brasilianisch-Bunt-2.jpg
def normalized(a, axis=-1, order=2):
    import numpy as np  # pylint: disable=import-outside-toplevel

    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2 == 0] = 1
    return a / np.expand_dims(l2, axis)


device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14", device=device)

image = preprocess(Image.open("_vyr_6097Sexy-Push-Up-Bikini-Brasilianisch-Bunt-2.jpg")).unsqueeze(0).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)

emb = np.asarray( normalized(image_features.detach().cpu()))

nsfw_value = safety_model.predict(emb)

print(nsfw_value)

--2022-12-06 05:58:40--  https://www.damenmode-quelle.de/fotky421/fotos/_vyr_6097Sexy-Push-Up-Bikini-Brasilianisch-Bunt-2.jpg
Resolving www.damenmode-quelle.de (www.damenmode-quelle.de)... 81.0.206.116, 2001:1528:114::661c
Connecting to www.damenmode-quelle.de (www.damenmode-quelle.de)|81.0.206.116|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 175773 (172K) [image/jpeg]
Saving to: ‘_vyr_6097Sexy-Push-Up-Bikini-Brasilianisch-Bunt-2.jpg’

_vyr_6097Sexy-Push- 100%[===================>] 171.65K   911KB/s    in 0.2s    

2022-12-06 05:58:40 (911 KB/s) - ‘_vyr_6097Sexy-Push-Up-Bikini-Brasilianisch-Bunt-2.jpg’ saved [175773/175773]



100%|████████████████████████████████████████| 933M/933M [00:04<00:00, 217MiB/s]


1/1 [==============================] - 7s 7s/step
[[0.99981856]]
